In [1]:
import cv2 as cv
import os
import shutil
import random
import numpy as np
from PIL import Image

**补充图片

In [2]:
def Fill(src_file):#src_file = './dataset/正例'
    filename = os.listdir(src_file)
    for i in filename:
        n = os.listdir(src_file+'/'+i)
        if len(n)<5:
            n1 = set([n[j].split('--')[1] for j in range(len(n))])
            n2 = set(['1.jpg','2.jpg','3.jpg','4.jpg','5.jpg'])
            n3 = list(n2 - n1)
            for j in n3:
                k = 0
                while(k<1):
                    file = random.sample(filename,1)
                    images = os.listdir(src_file+'/' + file[0])
                    for im in images:
                        if im.endswith(j):
                            shutil.copyfile(src_file+'/'+file[0]+'/'+im, src_file+'/'+i+'/'+im)
                            k = 1

In [ ]:
Fill('./dataset/正例')

In [4]:
Fill('./dataset/负例/ASD')

In [5]:
Fill('./dataset/负例/VSD')

**裁剪扇形图片

In [35]:
def crop(img):
    # 绘制扇形  1.目标图片  2.椭圆圆心  3.长短轴长度  4.偏转角度  5.圆弧起始角度  6.终止角度  7.颜色  8.是否填充
    dst = np.zeros([768, 1024])
    x = cv.ellipse(dst,(530,125),(550,550),0,48,132,255,-1)/255
    img = img*x
    img = img.astype('uint8')
    return img

In [39]:
def Tailoring(src_file):#src_file = './dataset/test/正例'
    filename = os.listdir(src_file)
    for file in filename:
        for img in os.listdir(src_file+'/'+file):
            if img != '.ipynb_checkpoints':
                path = src_file+'/'+file+'/'+img
                #print(path)
                img = cv.imread(path,cv.IMREAD_GRAYSCALE)
                if img.shape != (768, 1024):
                    img = cv.resize(img,(1024,768))
                img = crop(img)
                cv.imwrite(path, img)    

In [41]:
Tailoring('./dataset 5c/Negative/VSD')

**修改文件名

In [48]:
def rename(src_file):#src_file = './dataset/正例'
    filename = os.listdir(src_file)
    for i in filename:
        n = os.listdir(src_file+'/'+i)
        for j in n:
            if len(j.split('--'))>1:
                os.rename(src_file+'/'+i+'/'+j,src_file+'/'+i+'/'+j.split('--')[1])

In [54]:
rename('./dataset/test/正例')

**删除dataset中ipynb_checkpoints文件

In [30]:
def rm_ipynb_checkpoints(src_file):
    filename = os.listdir(src_file)
    for file in filename:
        if file == '.ipynb_checkpoints':
            shutil.rmtree(src_file+'/'+file)
            print(0)
        else:
            for img in os.listdir(src_file+'/'+file):
                if img == '.ipynb_checkpoints':
                    print(1)
                    shutil.rmtree(src_file+'/'+file+'/'+img)

In [32]:
rm_ipynb_checkpoints('./dataset/test/正例')
rm_ipynb_checkpoints('./dataset/test/负例/ASD')
rm_ipynb_checkpoints('./dataset/test/负例/VSD')
rm_ipynb_checkpoints('./dataset/train/正例')
rm_ipynb_checkpoints('./dataset/train/负例/ASD')
rm_ipynb_checkpoints('./dataset/train/负例/VSD')

1


**矩形裁剪

In [42]:
def crop_M(src_file):
    filename = os.listdir(src_file)
    for file in filename:
        for img in os.listdir(src_file+'/'+file):
            image = Image.open(src_file+'/'+file+'/'+img)
            image = image.crop([230,150,830,650])
            image.save(src_file+'/'+file+'/'+img)

In [43]:
crop_M('./dataset 5c/Positive')
crop_M('./dataset 5c/Negative/ASD')
crop_M('./dataset 5c/Negative/VSD')

**从视频中抽取A4C图像

In [2]:
def extract_img(root_file):
    paths = []
    for r,d,f in os.walk(root_file):
        for i in d:
            if i == '四腔心二维动态':
                paths.append(r+'/'+i)
    for p in paths:
        filename = os.listdir(p)
        for f in filename:
            cap = cv.VideoCapture(os.path.join(p,f))
            s = 0
            while(True):
                ret, frame = cap.read()
                if ret == True:
                    s+=1
                else:
                    break
                if s%10==0:
                    cv.imwrite('./dataset2/Negative/VSD/'+f.split('.avi')[0]+'_'+str(s//10)+'.jpg',frame)
            

In [24]:
extract_img('../先心病数据二/二维动态/正常')

In [26]:
extract_img('../先心病数据二/二维动态/ASD')

In [28]:
extract_img('../先心病数据二/二维动态/VSD')

**将dataset1中的A4C和合并到dataset2中

In [44]:
def sum_A4Cimg(src_file):
    filename = os.listdir(src_file)
    for f in filename:
        for img in os.listdir(os.path.join(src_file, f)):
            print(img)
            if img.split('--')[1]=='3.jpg':
                shutil.copy(src_file+'/'+f+'/'+img, './dataset2/Positive/'+img)

In [ ]:
sum_A4Cimg('./dataset原版/1最终版本/正例')

**从视频中抽取5张图片

In [34]:
paths = []
for r,d,f in os.walk('./二维动态/VSD'):
    if '四腔心二维动态' in d:
        paths.append(r)
f = 0
for path in paths:
    for i in os.listdir(os.path.join(path, '四腔心二维动态' )):
        cap1 = cv.VideoCapture(os.path.join(os.path.join(path, '四腔心二维动态' ),i))
        #print(os.path.join(path, '四腔心二维动态' ),i)
        D = int(i[-21:-4])
        v = []
        ff = ['大动脉短轴二维动态','剑下双房二维动态','主动脉弓长轴二维动态','左室长轴二维动态']
        for j in ff:
            min_D = 10000000000
            for k in os.listdir(os.path.join(path, j )):
                #print(os.path.join(path, j ),k)
                if abs(int(i[-21:-4])-int(k[-21:-4]))<min_D:
                    min_D = abs(int(i[-21:-4])-int(k[-21:-4]))
                    k1 = k
            v.append(k1)
        cap2 = cv.VideoCapture(os.path.join(os.path.join(path, ff[0] ),v[0]))
        cap3 = cv.VideoCapture(os.path.join(os.path.join(path, ff[1] ),v[1]))
        cap4 = cv.VideoCapture(os.path.join(os.path.join(path, ff[2] ),v[2]))
        cap5 = cv.VideoCapture(os.path.join(os.path.join(path, ff[3] ),v[3]))
        #print(v)
        s = 0
        while(True):
            ret1, frame1 = cap2.read()
            ret2, frame2 = cap3.read()
            ret3, frame3 = cap1.read()
            ret4, frame4 = cap4.read()
            ret5, frame5 = cap5.read()   
            if (ret1,ret2,ret3,ret4,ret5) == (True,True,True,True,True):
                s+=1
            else:
                break
            if s%10==0:
                f += 1
                os.mkdir('./dataset 5c/Negative/VSD/'+str(f))
                cv.imwrite('./dataset 5c/Negative/VSD/'+str(f)+'/1.jpg',frame1)
                cv.imwrite('./dataset 5c/Negative/VSD/'+str(f)+'/2.jpg',frame2)
                cv.imwrite('./dataset 5c/Negative/VSD/'+str(f)+'/3.jpg',frame3)
                cv.imwrite('./dataset 5c/Negative/VSD/'+str(f)+'/4.jpg',frame4)
                cv.imwrite('./dataset 5c/Negative/VSD/'+str(f)+'/5.jpg',frame5)

In [2]:
import shutil
import os
for r,d,f in os.walk(('./')):
    if '.ipynb_checkpoints' in d:
        shutil.rmtree(r+'/.ipynb_checkpoints')
        print(1)

1
1
1
1
1
1
1
1
1
1


**将数据集分为train、verify和test

In [4]:
f1 = os.listdir('./dataset 5c/Positive')
f2 = os.listdir('./dataset 5c/Negative/ASD')
f3 = os.listdir('./dataset 5c/Negative/VSD')

In [5]:
print(len(f1),len(f2),len(f3))

3856 891 1270


In [6]:
n1 = np.arange(3856)
n2 = np.arange(891)
n3 = np.arange(1270)
np.random.shuffle(n1)
np.random.shuffle(n2)
np.random.shuffle(n3)

In [10]:
for i in n1[0:385]:
    shutil.move('./dataset 5c/Positive/'+f1[i],'./dataset 5c/verify/Positive/'+f1[i])
for i in n1[385:770]:
    shutil.move('./dataset 5c/Positive/'+f1[i],'./dataset 5c/test/Positive/'+f1[i])

In [12]:
for i in n2[0:89]:
    shutil.move('./dataset 5c/Negative/ASD/'+f2[i],'./dataset 5c/verify/Negative/ASD/'+f2[i])
for i in n2[89:178]:
    shutil.move('./dataset 5c/Negative/ASD/'+f2[i],'./dataset 5c/test/Negative/ASD/'+f2[i])

In [13]:
for i in n3[0:127]:
    shutil.move('./dataset 5c/Negative/VSD/'+f3[i],'./dataset 5c/verify/Negative/VSD/'+f3[i])
for i in n3[127:254]:
    shutil.move('./dataset 5c/Negative/VSD/'+f3[i],'./dataset 5c/test/Negative/VSD/'+f3[i])